In [1]:
from itertools import chain
import nltk
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import sklearn
import pycrfsuite
import pandas as pd

# Import csv from preprocessing - generated with python script

In [2]:
test = pd.read_csv('../test.csv')

## Remove lexsyn feature - not included yet

In [3]:
test = test.loc[:, test.columns != 'lexsyn']


## Preprocess data from csv/DataFrame

In [4]:
sent_x = []
sent_y = []
for p in set(test.paragraph.values):
    for s in set(test[test.paragraph==p].sentence.values):
        temp_test = test[(test.paragraph == p) & (test.sentence == s)]
        sent_x.append(temp_test.loc[:, temp_test.columns != 'IOB'].to_dict("record"))
        sent_y.append(temp_test.IOB.values)

/var/folders/7r/c0g604c13xn1wrqn69g540c00000gn/T/ipykernel_42961/1349379417.py:6: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  sent_x.append(temp_test.loc[:, temp_test.columns != 'IOB'].to_dict("record"))


## Split data - informal for now and only on esssay001

In [5]:
X_train = sent_x[:10]
y_train = sent_y[:10]
X_test = sent_x[10:]
y_test = sent_y[10:]

## Initialize CRF trainer

In [6]:
trainer = pycrfsuite.Trainer(verbose=False)


### add training data

In [7]:
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

In [8]:
trainer.set_params({
    'c1': 1.0,   # coefficient for L1 penalty
    'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': 50,  # stop earlier

    # include transitions that are possible, but not observed
    'feature.possible_transitions': True
})

## Train and save to file

In [9]:
trainer.train('conll2002-esp.crfsuite')

## Load model from file

In [10]:
tagger = pycrfsuite.Tagger()
tagger.open('conll2002-esp.crfsuite')

## Test

In [14]:
print("Predicted:", ' '.join(tagger.tag(X_test[2])))
print("Correct:  ", ' '.join(y_test[2]))

Predicted: Arg-B Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I
Correct:   Arg-B Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I Arg-I O
